In [4]:
import torch
import torch.nn as nn

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
torch.manual_seed(42)

In [7]:
class NeuralNetwork(nn.Module):
    input_dim = 2
    output_dim = 2
    
    def __init__(self, num_layers, neurons_per_layer):
        
        super().__init__()
        self.layers = nn.ModuleList()
        current_dim = self.input_dim
        
        for layer in range(num_layers):
            l = nn.Linear(current_dim, neurons_per_layer)
            nn.init.xavier_normal_(l.weight)
            nn.init.zeros_(l.bias)
            self.layers.append(l)
            current_dim = neurons_per_layer
        
        self.layers.append(nn.Linear(current_dim, self.output_dim))
        
        self.bnLayer = nn.ModuleList()
        for layer in range(num_layers):
            self.bnLayer.append(nn.BatchNorm1d(neurons_per_layer))
            
    def forward(self, x, fixedIdx):
        m = nn.ReLU6()
        ctr = 0
        
        xv = x[:, 0]
        yv = x[:, 1]
        
        x = torch.transpose(torch.stack((xv, yv)), 0, 1)
        
        for layer in self.layers[:-1]:
            x = m(self.bnLayer[crt](layer(x)))
            ctr += 1
        
        out = 0.01 + torch.softmax(self.layers[-1](x), dim = 1)
        rho = out[:,0].view(-1);

        rho = (1-fixedIdx)*rho + fixedIdx*(rho + torch.abs(1-rho))

In [10]:
neural_network = NeuralNetwork(5, 20)
print(neural_network)

NeuralNetwork(
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=20, bias=True)
    (2): Linear(in_features=20, out_features=20, bias=True)
    (3): Linear(in_features=20, out_features=20, bias=True)
    (4): Linear(in_features=20, out_features=20, bias=True)
    (5): Linear(in_features=20, out_features=2, bias=True)
  )
  (bnLayer): ModuleList(
    (0): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)


In [26]:
class NeuralNetworkMn(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.layers = nn.Sequential(
            #nn.Flatten(),
            nn.Linear(2, 20),
            nn.BatchNorm1d(20),
            nn.ReLU(),
            nn.Linear(20, 20),
            nn.BatchNorm1d(20),
            nn.ReLU(),
            nn.Linear(20, 20),
            nn.BatchNorm1d(20),
            nn.ReLU(),
            nn.Linear(20, 20),
            nn.BatchNorm1d(20),
            nn.ReLU(),
            nn.Linear(20, 20),
            nn.BatchNorm1d(20),
            nn.ReLU(),
            nn.Linear(20, 2),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x, fixedIdx):
        
        xv = x[:, 0]
        yv = x[:, 1]
        
        x = torch.transpose(torch.stack((xv, yv)), 0, 1)
        
        out = 0.01 + self.layers(x)
        rho = out[:,0].view(-1)
        
        return (1-fixedIdx)*rho + fixedIdx*(rho + torch.abs(1-rho))
        
        

In [27]:
nn_mn = NeuralNetworkMn()
print(nn_mn)

NeuralNetworkMn(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=20, bias=True)
    (1): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=20, out_features=20, bias=True)
    (4): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=20, out_features=20, bias=True)
    (7): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Linear(in_features=20, out_features=20, bias=True)
    (10): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Linear(in_features=20, out_features=20, bias=True)
    (13): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Linear(in_features=20, out_features=2, bias=True)
    (16): Softmax(dim=1)
  )
)
